In [1]:
from pathlib import Path

import pandas as pd
from db import *

/home/jovyan/work/db.py:8: SAWarning: Did not recognize type 'geometry' of column 'geom'
  mapping_hotel = Table(


In [2]:
base_dir = Path(".").resolve()

In [3]:
room_mapping = pd.read_csv(base_dir / "data" / "room_mapping.csv")

In [5]:
room_mapping.shape

(19654, 40)

In [ ]:
hotel_mapping.head()

In [ ]:
id_mapping = dict(zip(hotel_mapping["hotel_name"], hotel_mapping["hotel_id"]))

In [ ]:
room_mapping["hotel_id"] = room_mapping["HotelName"].map(id_mapping)

In [ ]:
room_mapping.head()

In [ ]:
room_mapping.drop_duplicates(
    subset=["hotel_id", "MapRoomCode"], keep="last", inplace=True
)

In [ ]:
import re

replacement_mapping = {
    r"Single Use": " ",
    r"Single": " ",
    r"Singe": " ",
    r"\( \)": " ",
    r"Twin Rm/": "Twin Room",
    r"Bedrooms": "Bedroom",
    r"01": "1",
    r"02": "2",
    r"03": "3",
}


def replace_strings(s):
    for pattern, replacement in replacement_mapping.items():
        s = re.sub(pattern, replacement, s, flags=re.IGNORECASE)
    return s.strip()


room_mapping["HotelRoomDescription"] = room_mapping["HotelRoomDescription"].apply(
    replace_strings
)
room_mapping["HotelRoomDescription"] = (
    room_mapping["HotelRoomDescription"].str.split().str.join(" ").str.title()
)

In [ ]:
room_mapping_unique = (
    room_mapping.groupby(["HotelRoomDescription", "hotel_id"]).size().reset_index()
)
room_mapping_unique.sort_values(["hotel_id", "HotelRoomDescription"], inplace=True)
room_mapping_unique.to_csv("room_mapping_unique.csv", index=False)

In [ ]:
room_mapping.to_csv("room_mapping.csv", index=False)

In [ ]:
room_mapping_unique.to_csv("room_mapping_unique.csv", index=False)

In [ ]:
import json

output = []

for _, row in room_mapping_unique.iterrows():
    hotel_room_dict = {
        "model": "accommodation.hotelroom",
        "fields": {
            "hotel": row["hotel_id"],
            "name": row["HotelRoomDescription"],
        },
    }
    output.append(hotel_room_dict)

output_json = json.dumps(output, indent=2, default=str)

with open(base_dir / "fixtures" / "hotelroom.json", "w") as f:
    f.write(output_json)

In [ ]:
accommodation_hotel_room = pd.read_sql(
    sql=select(accommodation_hotel_room), con=engine.connect()
)

In [ ]:
accommodation_hotel_room.head()

In [ ]:
id_mapping = dict(
    zip(
        accommodation_hotel_room.apply(lambda x: (x["name"], x["hotel_id"]), axis=1),
        accommodation_hotel_room["id"],
    )
)

In [ ]:
room_mapping["room_id"] = room_mapping.apply(
    lambda x: id_mapping.get(
        (
            x["HotelRoomDescription"],
            x["hotel_id"],
        )
    ),
    axis=1,
)

In [ ]:
room_mapping.dropna(subset=["hotel_id", "room_id"], inplace=True)
room_mapping["room_type"] = room_mapping["RoomDescription"].str.cat(
    room_mapping["RoomView"].fillna(""), sep=" "
)

In [ ]:
output = []

for _, row in room_mapping.iterrows():
    hotel_room_mapping_dict = {
        "model": "mapping.hotelroommapping",
        "fields": {
            "hotel_room": row["room_id"],
            "room_type": row["room_type"],
            "room_code": row["MapRoomCode"],
        },
    }
    output.append(hotel_room_mapping_dict)

output_json = json.dumps(output, indent=2, default=str)

with open("fixtures/hotelroommapping.json", "w") as f:
    f.write(output_json)

In [ ]:
room_mapping.to_csv("room_mapping.csv", index=False)